In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import msgpack
from state import XYMStateMap

import networkx as nx 
from pyvis.network import Network
import plotly.graph_objects as go
from tqdm.notebook import tqdm

import dash
import jupyter_dash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State

cyto.load_extra_layouts()


In [2]:
state_map = XYMStateMap.read_msgpack('./state_map.msgpack')

In [ ]:
MIN_NODE = 5 # smallest node you want to see rendered in terms of blocks harvested
MIN_DELEGATE = 5 # smallest delegate you want to see rendered in terms of blocks harvested
SCALE = 1000 # starting scale to get a decent layout for the fixed points

# hack to fix the positions of "important" nodes seeing as layout is too expensive to compute on the fly
node_graph = state_map.get_harvester_graph(min_node_size=MIN_NODE*100,min_delegate_size=MIN_DELEGATE*10)
pos = nx.spring_layout(node_graph,scale=SCALE,center=np.zeros(2),weight=None)
elements = nx.readwrite.json_graph.cytoscape_data(node_graph)['elements']
for node in elements['nodes']:
    node['position'] = {'x':pos[node['data']['id']][0],'y':pos[node['data']['id']][1]}


In [43]:
app = jupyter_dash.JupyterDash(__name__)


default_stylesheet = [
    {
        "selector": "node",
        "style": {
            "width": "mapData(size, 1, 100, 15, 150)",
            "height": "mapData(size, 1, 100, 15, 150)",
            "content": "data(size)",
            "font-size": "12px",
            "text-valign": "center",
            "text-halign": "center",
            "background-color": "data(color)",
            "border-color": "black",
            "border-width": 1,
            "opacity": "mapData(size,0,1,0,1)",
        }},
    {
        "selector": "edge",
        "style": {
            "width": 2,
            "curve-style": "straight",
            "color": "black",
            "opacity": "mapData(weight, 0, 5, 0.0, 1.0)",
        }
    }
]


app.layout = html.Div([

    cyto.Cytoscape(
        id='cytoscape-harvesters',
        layout={
            'name': 'preset',
        },
        style={'width': '100%', 'height': '95vh'},
        stylesheet=default_stylesheet,
        elements=elements,
        responsive=True
    ),
    dcc.Slider(
        id='height-center-slider',
        min=0,
        max=max([max(x['xym_balance'].keys()) for x in state_map.values() if len(x['xym_balance'])]),
        step=100,
        value=1000,
        tooltip={'always_visible':True,'placement':'bottom'}
    ),  
    dcc.Slider(
        id='height-range-slider',
        min=100,
        max=10000,
        step=100,
        value=1000,
        tooltip={'always_visible':True,'placement':'bottom'}
    ),  
])


@app.callback(Output('cytoscape-harvesters', 'elements'),
              Input('height-center-slider', 'value'),
              Input('height-range-slider', 'value'),
              State('cytoscape-harvesters', 'elements'))
def update_elements(height_center,height_range,elements):
    if height_center is not None and height_range is not None:
        node_graph = state_map.get_harvester_graph(height_center-height_range,height_center+height_range,min_node_size=MIN_NODE,min_delegate_size=MIN_DELEGATE)

        fixed_keys = pos.keys()
        for node in node_graph.nodes():
            if node not in fixed_keys:
                pos[node] = np.random.normal(0,1/3,size=[2]) * SCALE
        
        new_pos = nx.spring_layout(node_graph,k=0.01,pos=pos,fixed=[x for x in fixed_keys if x in node_graph],weight=None)  
        
        for n,p in new_pos.items():
            if n not in fixed_keys:
                pos[n] = p
    
        elements = nx.readwrite.json_graph.cytoscape_data(node_graph)['elements']
        for node in elements['nodes']:
            node['position'] = {'x':pos[node['data']['id']][0],'y':pos[node['data']['id']][1]}

    return elements


# @app.callback(Output('cytoscape-harvesters', 'elements'),
#               Input('height-start-slider', 'value'),
#               Input('height-end-slider', 'value'),
#               State('cytoscape-harvesters', 'elements'))
# def update_elements(height_start,height_end,elements):
    
#     if height_start is not None and height_end is not None:
        
#         node_graph = state_map.get_harvester_graph(height_start,height_end)

#         for n in elements['nodes']:
#             if n['data']['id'] in node_graph:
#                 n['data']['size'] = node_graph.nodes()[n['data']['id']]['size']
#             else:
#                 n['data']['size'] = 0

#         for e in elements['edges']:
#             if (e['data']['source'],e['data']['target']) in node_graph.edges():
#                 e['data']['weight'] = node_graph.edges()[(e['data']['source'],e['data']['target'])]['weight']
#             else:
#                 e['data']['weight'] = 0
            
#     return elements


app.run_server(mode='external', host='0.0.0.0', port=8050, threaded=True, debug=True)

Dash app running on http://0.0.0.0:8050/
